In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
import loadOSCdata
import pandas as pd
import numpy as np

In [2]:
#idNotWork = 292590
idWork=219598

In [3]:
geoffTrip = 401439

In [ ]:
fotos = loadOSCdata.queryOSCapi(OSCid = geoffTrip, output = 'csv', outputFile = 'photos.csv',dataType='photos')
fotos.head()

In [ ]:
acce = loadOSCdata.queryOSCapi(OSCid = idWork, output = 'csv', outputFile = 'acce.csv',dataType='accelerometer')
acce.head()

In [ ]:
#acce

##tripid
##change pointIndex to Point iD

#photo
##pointID
##tripid
##timestamp

In [ ]:
fotos.pictureName[0]

In [ ]:
data = loadOSCdata.getOSCjson(OSCid=geoffTrip)

In [ ]:
data = data['osv']

In [ ]:
OSCid = 401439

In [ ]:
totalPhotos = len(data['photos'])


pictureName = [data['photos'][i]['name'] for i in range(totalPhotos)]
quality = np.repeat(np.nan,totalPhotos)
latitude = [data['photos'][i]['lat'] for i in range(totalPhotos)]
longitude = [data['photos'][i]['lng'] for i in range(totalPhotos)]
timestamp = [data['photos'][i]['date_added'] for i in range(totalPhotos)]


outputReturn = pd.DataFrame({'pictureName':pictureName,
                                     'quality':quality,
                                     'latitude':latitude,
                                     'longitude':longitude,
                            'timestamp':timestamp})


#photo
##pointID
##tripid
##timestamp
outputReturn['tripID'] = OSCid
outputReturn['pointID'] = outputReturn.index

if not(os.path.exists("../data")):
    os.system('mkdir data')
    
if not(os.path.exists('../data/' + str(OSCid))):
    os.system('mkdir ../data/' + str(OSCid))
            
#download data
for i in range(outputReturn.shape[0]):
    photoURL = outputReturn.pictureName[i][9:]
    oscURL = 'http://'+outputReturn.pictureName[0][0:8]+'.openstreetcam.org/'
    os.system('wget '+ oscURL + photoURL + ' -P ' + '../data/' + str(OSCid))

In [ ]:
outputReturn.head()

In [ ]:
outputReturn.pictureName[0][0:8]

In [ ]:
#check if directories exists
